In [181]:
import helpers
import numpy as np
import re
import math
from tqdm import tqdm
import copy
import networkx as nx
from collections import Counter

data_hex = """A0016C880162017C3686B18A3D4780"""
#data_hex = "D2FE28"

with open("input_16.txt", 'r') as f:
    data_hex = f.read()
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = re.findall("(\d+),(\d+) -> (\d+),(\d+)", data)
#data = [x.rstrip() for x in data.split("\n\n") if x]


data_hex = data_hex.lstrip()
data_hex = data_hex.rstrip()
#data = np.array([np.array([int(y) for y in x]) for x in data.split("\n")])

#data = [x.split("-") for x in data.split("\n")]

data_hex

'8054F9C95F9C1C973D000D0A79F6635986270B054AE9EE51F8001D395CCFE21042497E4A2F6200E1803B0C20846820043630C1F8A840087C6C8BB1688018395559A30997A8AE60064D17980291734016100622F41F8DC200F4118D3175400E896C068E98016E00790169A600590141EE0062801E8041E800F1A0036C28010402CD3801A60053007928018CA8014400EF2801D359FFA732A000D2623CADE7C907C2C96F5F6992AC440157F002032CE92CE9352AF9F4C0119BDEE93E6F9C55D004E66A8B335445009E1CCCEAFD299AA4C066AB1BD4C5804149C1193EE1967AB7F214CF74752B1E5CEDC02297838C649F6F9138300424B9C34B004A63CCF238A56B71520142A5A7FC672E5E00B080350663B44F1006A2047B8C51CC80286C0055253951F98469F1D86D3C1E600F80021118A124261006E23C7E8260008641A8D51F0C01299EC3F4B6A37CABD80252211221A600BC930D0057B2FAA31CDCEF6B76DADF1666FE2E000FA4905CB7239AFAC0660114B39C9BA492D4EBB180252E472AD6C00BF48C350F9F47D2012B6C014000436284628BE00087C5D8671F27F0C480259C9FE16D1F4B224942B6F39CAF767931CFC36BC800EA4FF9CE0CCE4FCA4600ACCC690DE738D39D006A000087C2A89D0DC401987B136259006AFA00ACA7DBA53EDB31F9F3DBF31900559C00BCCC4936473A639A5

In [182]:
data_bin = "{0:08b}".format(int(data_hex, 16)).zfill(len(data_hex)*4)
data_bin
len(data_bin)

5464

In [183]:
def parse(s, num=None):
    out_l = []
    if num:
        while num > 0:
            out, s = parse_1(s)
            out_l.append(out)
            num -= 1
    else:
        while len(s) > 0 and not s.count("0") == len(s):
            print(s, len(s) > 0, s.count("0") == len(s))
            out, s = parse_1(s)
            out_l.append(out)
    return out_l, s
        

def parse_1(s):
    print(s)
    bits, s = getBits(s, 3)
    print(bits)
    package_version = int(bits, 2)
    bits, s = getBits(s, 3)
    package_type = int(bits, 2)
    print(package_version, package_type)
    if package_type == 4:
        val, s = parseLiteral(s)
        return [package_version, package_type, val], s
    elif not package_type == 4:
        bits, s = getBits(s, 1)
        print(f'length type ID bits {bits}')
        if bits == "0":
            bits, s = getBits(s, 15)
            bits_len = int(bits, 2)
            print(f'length bits {bits_len}')
            bits, s = getBits(s, bits_len)
            return [package_version, package_type, parse(bits)[0]], s
        elif bits == "1":
            bits, s = getBits(s, 11)
            num_package = int(bits, 2)
            print(f'num package {num_package}')
            outp, s = parse(s, num_package)
            return [package_version, package_type, outp], s
        
def getBits(d, n):
    return d[:n], d[n:]

def parseLiteral(s):
    parsing = True
    val = ""
    while parsing:
        group, s = getBits(s, 5)
        if group[0] == "0":
            parsing = False
        val += group[1:]
    return int(val, 2), s

In [184]:
s = copy.deepcopy(data_bin)
sol, s = parse(s)
sol

1000000001010100111110011100100101011111100111000001110010010111001111010000000000001101000010100111100111110110011000110101100110000110001001110000101100000101010010101110100111101110010100011111100000000000000111010011100101011100110011111110001000010000010000100100100101111110010010100010111101100010000000001110000110000000001110110000110000100000100001000110100000100000000001000011011000110000110000011111100010101000010000000000100001111100011011001000101110110001011010001000000000011000001110010101010101011001101000110000100110010111101010001010111001100000000001100100110100010111100110000000001010010001011100110100000000010110000100000000011000100010111101000001111110001101110000100000000011110100000100011000110100110001011101010100000000001110100010010110110000000110100011101001100000000001011011100000000001111001000000010110100110100110000000000101100100000001010000011110111000000000011000101000000000011110100000000100000111101000000000001111000110100000000000110110110000101000

[[4,
  0,
  [[3, 4, 163011],
   [4, 4, 119],
   [5, 0, [[2, 4, 239], [5, 4, 37195624545], [2, 4, 1455], [3, 4, 79]]],
   [6, 0, [[3, 4, 365554], [0, 4, 2], [0, 4, 43]]],
   [7, 4, 543],
   [3,
    0,
    [[4, 1, [[5, 4, 6], [0, 4, 2], [0, 4, 13]]],
     [0, 1, [[5, 4, 12], [1, 4, 3], [7, 4, 10]]],
     [4, 1, [[7, 4, 13], [4, 4, 11], [5, 4, 11]]]]],
   [2, 1, [[3, 4, 90], [5, 4, 166], [0, 4, 159]]],
   [2, 4, 10],
   [7, 1, [[4, 4, 165]]],
   [7,
    1,
    [[4, 4, 11],
     [4,
      6,
      [[6, 0, [[0, 4, 11], [6, 4, 3], [7, 4, 13]]],
       [6, 0, [[6, 4, 2], [1, 4, 13], [1, 4, 11]]]]]]],
   [5, 2, [[6, 4, 9]]],
   [3,
    3,
    [[5,
      1,
      [[3,
        3,
        [[7,
          1,
          [[5,
            1,
            [[3,
              1,
              [[7,
                3,
                [[4,
                  2,
                  [[7,
                    2,
                    [[5,
                      0,
                      [[5,
                        0,
 

In [185]:
version_numbers = list()
def flatten_list(list_of_lists):
    for item in list_of_lists:
        if type(item) == list:
            if not type(item[0]) == list:
                version_numbers.append(item[0])
            flatten_list(item)
        else:
            flat_list.append(item)
    
    return flat_list

In [186]:
flatten_list(sol)
sum(version_numbers)

1007

In [134]:
sol

[[7, 3, [[2, 4, 1], [4, 4, 2], [1, 4, 3]]]]

In [135]:
sol_flat[0::2]

[7, 2, 1, 4, 1, 3]